# 杭州萧山区项目`WRF-CMAQ`模拟分析
## 模拟结果预处理：*`MET-Data`*

---
*@author: Evan*\
*@date: 2023-08-28*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd

import sys
sys.path.append('../../src/')
from namelist import *

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
chem = xr.open_dataset('D:/data/Project_Xiaoshan/202205/COMBINE_ACONC_v54_ISAM_intel_CN3XS_135X138_2022_chem.nc')
grid = xr.open_dataset('D:/data/Project_Xiaoshan/GRIDCRO2D_2022234.nc')

In [3]:
# 将时间点转换为实际时间
times=pd.date_range(timestart,timeend,freq='h')
times

DatetimeIndex(['2022-05-01 00:00:00', '2022-05-01 01:00:00',
               '2022-05-01 02:00:00', '2022-05-01 03:00:00',
               '2022-05-01 04:00:00', '2022-05-01 05:00:00',
               '2022-05-01 06:00:00', '2022-05-01 07:00:00',
               '2022-05-01 08:00:00', '2022-05-01 09:00:00',
               ...
               '2022-05-31 14:00:00', '2022-05-31 15:00:00',
               '2022-05-31 16:00:00', '2022-05-31 17:00:00',
               '2022-05-31 18:00:00', '2022-05-31 19:00:00',
               '2022-05-31 20:00:00', '2022-05-31 21:00:00',
               '2022-05-31 22:00:00', '2022-05-31 23:00:00'],
              dtype='datetime64[ns]', length=744, freq='H')

In [4]:
# 将层数转换为气压高度
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50
pres

array([1000.   ,  998.005,  995.82 ,  993.445,  990.88 ,  988.125,
        985.18 ,  981.665,  977.485,  972.545,  966.655,  959.625,
        951.36 ,  941.575,  929.985,  916.4  ,  900.345,  881.535,
        859.495,  833.845,  804.015,  771.715,  736.755,  703.885,
        658.95 ,  616.2  ,  570.98 ,  523.575,  474.365,  423.73 ,
        372.335,  320.75 ,  270.02 ,  221.095,  175.78 ,  135.785,
        101.49 ,   72.895])

In [5]:
# 结合地形高度HT与各气压层离地高度ZH获得各层实际海拔高度
ht=np.squeeze(grid.HT)
height=np.zeros([np.size(chem.ZH,0),
                 np.size(chem.ZH,1),
                 np.size(chem.ZH,2),
                 np.size(chem.ZH,3)])
for t in range(np.size(chem.ZH,0)):
    for l in range(np.size(chem.ZH,1)):
        height[t,l,:,:]=chem.ZH[t,l,:,:]+ht

In [6]:
days=1 # set spin-up days
dataset=xr.Dataset(
    data_vars=dict(
        # ! vars from CMAQ
        CO=(['time','level','y','x'],chem.CO[days*24-8:-8,:,:,:].data*30/22.4,{'long name':'Carbon Monoxide','units':'ug m-3'}),
        O3=(['time','level','y','x'],chem.O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Ozone','units':'ug m-3'}),
        NO=(['time','level','y','x'],chem.NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Nitric Oxide','units':'ug m-3'}),
        NO2=(['time','level','y','x'],chem.NO2[days*24-8:-8,:,:,:].data*37/22.4,{'long name':'Nitrogen Dioxide','units':'ug m-3'}),
        SO2=(['time','level','y','x'],chem.SO2[days*24-8:-8,:,:,:].data*64/22.4,{'long name':'Sulfur Dioxide','units':'ug m-3'}),
        VOC=(['time','level','y','x'],chem.VOC[days*24-8:-8,:,:,:].data,{'long name':'Volitile Organic Compounds','units':'ppbC'}),
        PM25=(['time','level','y','x'],chem.PM25_TOT[days*24-8:-8,:,:,:].data,{'long name':'','units':'ug m-3'}),
        PM25_NO3=(['time','level','y','x'],chem.PM25_NO3[days*24-8:-8,:,:,:].data,{'long name':'NO3 in PM25','units':'ug m-3'}),
        PM25_SO4=(['time','level','y','x'],chem.PM25_SO4[days*24-8:-8,:,:,:].data,{'long name':'SO4 in PM25','units':'ug m-3'}),
        PM10=(['time','level','y','x'],chem.PM10[days*24-8:-8,:,:,:].data,{'long name':'','units':'ug m-3'}),
        PMC_NO3=(['time','level','y','x'],chem.PMC_NO3[days*24-8:-8,:,:,:].data,{'long name':'NO3 in PMC','units':'ug m-3'}),
        PMC_SO4=(['time','level','y','x'],chem.PMC_SO4[days*24-8:-8,:,:,:].data,{'long name':'SO4 in PMC','units':'ug m-3'}),
        # ! wwind
        wwind=(['time','level','y','x'],chem.WWind[days*24-8:-8,:,:,:].data,{'long name':'Vertical Wind Speed','units':'m s-1'}),
        # ! altitude
        HT=(['time','level','y','x'],height[days*24-8:-8,:,:,:].data,{'long name':'Altitudes','units':'m'}),
        ),
    coords=dict(
        time=times,
        level=pres,
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Xiaoshan_202205',
        grid='CN3XS_135X138',
        createtime=pd.Timestamp.now().strftime('%Y-%m-%d'),
    ),
)
dataset

<xarray.Dataset>
Dimensions:    (time: 744, level: 38, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2022-05-01 ... 2022-05-31T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 135.8 101.5 72.89
    latitude   (y, x) float32 28.29 28.29 28.29 28.29 ... 31.89 31.89 31.89
    longitude  (y, x) float32 117.9 117.9 117.9 118.0 ... 122.3 122.3 122.4
Dimensions without coordinates: y, x
Data variables: (12/14)
    CO         (time, level, y, x) float32 400.6 390.4 387.1 ... 40.92 40.97
    O3         (time, level, y, x) float32 100.8 103.0 ... 1.636e+03 1.63e+03
    NO         (time, level, y, x) float32 0.08171 0.02923 ... 4.057e-09
    NO2        (time, level, y, x) float32 9.847 8.381 7.938 ... 0.1222 0.1225
    SO2        (time, level, y, x) float32 8.704 8.678 ... 0.004519 0.004535
    VOC        (time, level, y, x) float32 53.66 52.7 52.32 ... 0.015 0.01504
    ...         ...
    PM25_SO4   (time, level, y, x) float32 3.348 4.003 4.798 ... 0.0 0.0 0.0
    PM10       (time, level, y, x) float32 9.757 11.35 13.58 ... 0.0 0.0 0.0
    PMC_NO3    (time, level, y, x) float32 0.1797 0.2529 0.3322 ... 0.0 0.0 0.0
    PMC_SO4    (time, level, y, x) float32 0.03082 0.05153 0.07648 ... 0.0 0.0
    wwind      (time, level, y, x) float32 0.02042 0.03636 ... 0.007028 0.003645
    HT         (time, level, y, x) float64 115.2 203.0 ... 1.954e+04 1.954e+04
Attributes:
    case:        Xiaoshan_202205
    grid:        CN3XS_135X138
    createtime:  2023-08-28

In [7]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset.data_vars}
dataset.to_netcdf('D:/Download/XS_2205_chem.nc',encoding=encoding)